In [26]:
import numpy as np
import pandas as pd
import string
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
import nltk
import re
from nltk.stem import wordnet  
from nltk import pos_tag  

In [16]:
df = pd.read_csv('D:/MSc/SEMESTER 3/NLP/Chatbot/RickAndMortyScripts.csv')
df.sample(10)

,index,season no.,episode no.,episode name,name,line
1147,1448,3,1,The Rickshank Rickdemption,Summer,"Good roasted Cronenberg, I assume. Mom, you're..."
1526,1954,3,4,Vindicators 3 The Return of Worldender,Morty,I... think the personality conflict might have...
822,1019,2,5,Get Schwifty,Morty,"Rick, I think we need to cut our losses. We ge..."
591,671,2,1,A Rickle in Time,All Summers,What's happening Grampa?
1293,1676,3,3,Pickle Rick,Morty,Rick?
1115,1402,3,1,The Rickshank Rickdemption,Morty,Huh? What are you -- What are you doing?
1532,1960,3,4,Vindicators 3 The Return of Worldender,Morty,"Yeah, but not heroes."
372,424,1,2,Lawnmower Dog,Jerry,"Hey. Oh, wow. Okay, is is is everything okay i..."
479,551,1,6,Rick Potion 9,Rick,"Sorry, please proceed with your story about ba..."
1360,1745,3,3,Pickle Rick,Beth,He got wrapped up in an experiment. He's a sci...


In [18]:
df.shape

(1905, 6)

In [19]:
df.columns

Index(['index', 'season no.', 'episode no.', 'episode name', 'name', 'line'], dtype='object')

In [20]:
df = df.drop(columns=['index', 'season no.', 'episode no.', 'episode name'])
df.head()

,name,line
0,Rick,Morty! You gotta come on. Jus'... you gotta co...
1,Morty,"What, Rick? What’s going on?"
2,Rick,"I got a surprise for you, Morty."
3,Morty,It's the middle of the night. What are you tal...
4,Rick,"Come on, I got a surprise for you. Come on, h..."


In [21]:
df.isnull().values.any()

False

In [36]:
import re
import nltk
from nltk import pos_tag
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')

def text_normalization(text):
    text = str(text).lower()  # convert to all lower letters
    spl_char_text = re.sub(r'[^a-z]', ' ', text)  # remove any special characters including numbers
    tokens = nltk.word_tokenize(spl_char_text)  # tokenize words
    lema = WordNetLemmatizer()  # lemmatizer initiation
    tags_list = pos_tag(tokens, tagset=None)  # parts of speech
    lema_words = []
    for token, pos_token in tags_list:
        if pos_token.startswith('V'):  # if the tag from tag_list is a verb, assign 'v' to its pos_val
            pos_val = 'v'
        elif pos_token.startswith('J'):  # adjective
            pos_val = 'a'
        elif pos_token.startswith('R'):  # adverb
            pos_val = 'r'
        else:  # otherwise it must be a noun
            pos_val = 'n'
        lema_token = lema.lemmatize(token, pos_val)  # performing lemmatization
        lema_words.append(lema_token)  # adding the lemmatized words into our list
    return " ".join(lema_words)  # return our list as a human sentence

# Test the function
text = "This is a sample text for testing the function. It should work fine."
normalized_text = text_normalization(text)
print(normalized_text)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\noelm\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


this be a sample text for test the function it should work fine


In [37]:
import pandas as pd

# Assuming you already have your DataFrame df and the text normalization function text_normalization

# Apply text normalization function to each row of the DataFrame
normalized = df['line'].apply(text_normalization)

# Insert the normalized text into the DataFrame
df.insert(2, 'Normalized line', normalized, True)
df.head()


,name,line,Normalized line
0,Rick,Morty! You gotta come on. Jus'... you gotta co...,morty you get ta come on jus you get ta come w...
1,Morty,"What, Rick? What’s going on?",what rick what s go on
2,Rick,"I got a surprise for you, Morty.",i get a surprise for you morty
3,Morty,It's the middle of the night. What are you tal...,it s the middle of the night what be you talk ...
4,Rick,"Come on, I got a surprise for you. Come on, h...",come on i get a surprise for you come on hurry up


In [40]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
def removeStopWords(text):
    Q = []
    s = text.split()  # create an array of words from our text sentence, cut it into words
    q = ''
    for w in s:  # for every word in the given sentence if the word is a stop word ignore it
        if w in stop:
            continue
        else:  # otherwise add it to the end of our array
            Q.append(w)
        q = " ".join(Q)  # create a sentence out of our array of non stop words
    return q

In [41]:
normalized_non_stopwords = df['Normalized line'].apply(removeStopWords)
df.insert(3, 'Normalized and StopWords Removed', normalized_non_stopwords, True)
df.head()

,name,line,Normalized line,Normalized and StopWords Removed
0,Rick,Morty! You gotta come on. Jus'... you gotta co...,morty you get ta come on jus you get ta come w...,morty get ta come jus get ta come
1,Morty,"What, Rick? What’s going on?",what rick what s go on,rick go
2,Rick,"I got a surprise for you, Morty.",i get a surprise for you morty,get surprise morty
3,Morty,It's the middle of the night. What are you tal...,it s the middle of the night what be you talk ...,middle night talk
4,Rick,"Come on, I got a surprise for you. Come on, h...",come on i get a surprise for you come on hurry up,come get surprise come hurry


In [44]:
from sklearn.feature_extraction.text import TfidfVectorizer 

tfidf = TfidfVectorizer()  # initializing tf-idf
x_tfidf = tfidf.fit_transform(df['Normalized and StopWords Removed']).toarray()  # oversimplifying this converts words to vectors

features_tfidf = tfidf.get_feature_names_out()  # use function to get all the normalized words
df_tfidf = pd.DataFrame(x_tfidf, columns = features_tfidf)  # create dataframe to show the 0, 1 value for each word
# df_tfidf.loc[:,['hello', 'rick', 'morty']].head()  # show only specific columns
df_tfidf.head()

,aaaaaaaaaaaaaaaaaaaaaaaahhhhhhhhhh,aaaaaaaaaaaaaaaaaaaaaaahhhhhhhhhhhhhhhh,aaaaaaaaaaahhhhh,aaaaaaaahhhh,aaaaaaahhh,aaaaaaahhhh,aaaaaah,aaaaagh,aaaaah,aaaah,...,yo,young,youtube,yup,yyyyyyyyyyou,zero,zip,zit,zone,zorpathion
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [45]:

class Answer:
  def __init__(self, name, text):
    self.name = name
    self.text = text

In [49]:
from sklearn.metrics.pairwise import pairwise_distances
from sklearn.feature_extraction.text import TfidfVectorizer


def chat(question):
    tidy_question = text_normalization(removeStopWords(question))  # clean & lemmatize the question
    tf = tfidf.transform([tidy_question]).toarray()  # convert the question into a vector
    cos = 1- pairwise_distances(df_tfidf, tf, metric = 'cosine')  # calculate the cosine value
    index_value = cos.argmax()  # find the index of the maximum cosine value
    answer = Answer(df['name'].loc[index_value + 1], df['line'].loc[index_value + 1])
    return answer

In [50]:
def chatbot():
    exit_chatbot = False
    first_loop = True

    while exit_chatbot == False:
        if (first_loop):
            print("Welcome to the Rick and Morty chatbot! Type q to close it, otherwise let's get schwifty \U0001F44D")
            first_loop = False
        user_input_question = input("What do you want to say:")
            
        if(user_input_question.lower() == 'q'): 
            exit_chatbot = True
            print("Byeeee!")
        else :
            answer = chat(user_input_question)
            print(answer.name + "'s answer: " + answer.text)
            

In [51]:
chatbot()

Welcome to the Rick and Morty chatbot! Type q to close it, otherwise let's get schwifty 👍
Morty's answer: Whoa! Hi, Jessica.
Morty's answer: Ah, Jessica.
Jerry's answer: I tell ya, the Galactic Federation taking over Earth -- best thing that's ever happened to this family. I just got my sixth promotion this week, and I still don't know what I do.
Jerry's answer: I tell ya, the Galactic Federation taking over Earth -- best thing that's ever happened to this family. I just got my sixth promotion this week, and I still don't know what I do.
Snuffles's answer: Jerry, come to rub my face in urine again?
Jerry's answer: I tell ya, the Galactic Federation taking over Earth -- best thing that's ever happened to this family. I just got my sixth promotion this week, and I still don't know what I do.
Mr. Goldenfold's answer: Five plus five.
Morty's answer: 1.77245385... Whoa!
Morty's answer: What the hell? Why would Mr. Goldenfold's dream version of Mrs.
Byeeee!
